In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import pandas_profiling
from sklearn.model_selection import train_test_split

In [2]:
cf_train=pd.read_csv('D:/Aks/Project/Counterfeit/counterfeit_train.csv')
cf_test=pd.read_csv('D:/Aks/Project/Counterfeit/counterfeit_test.csv')

In [3]:
cf_test1=cf_test.copy()

In [4]:
pandas_profiling.ProfileReport(cf_train)

D:\Anaconda\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,12
Number of observations,6818
Total Missing (%),1.4%
Total size in memory,639.3 KiB
Average record size in memory,96.0 B
Numeric,5
Categorical,7
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [5]:
cf_train.columns

Index(['Medicine_ID', 'Counterfeit_Weight', 'DistArea_ID', 'Active_Since',
       'Medicine_MRP', 'Medicine_Type', 'SidEffect_Level',
       'Availability_rating', 'Area_Type', 'Area_City_Type', 'Area_dist_level',
       'Counterfeit_Sales'],
      dtype='object')

In [6]:
cf_test.columns

Index(['Medicine_ID', 'Counterfeit_Weight', 'DistArea_ID', 'Active_Since',
       'Medicine_MRP', 'Medicine_Type', 'SidEffect_Level',
       'Availability_rating', 'Area_Type', 'Area_City_Type',
       'Area_dist_level'],
      dtype='object')

In [7]:
cf_train.isnull().sum()

Medicine_ID               0
Counterfeit_Weight     1166
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales         0
dtype: int64

In [8]:
cf_test.isnull().sum()

Medicine_ID              0
Counterfeit_Weight     297
DistArea_ID              0
Active_Since             0
Medicine_MRP             0
Medicine_Type            0
SidEffect_Level          0
Availability_rating      0
Area_Type                0
Area_City_Type           0
Area_dist_level          0
dtype: int64

In [9]:
cf_train['Counterfeit_Weight'].fillna(cf_train['Counterfeit_Weight'].mean(),inplace=True)
cf_test['Counterfeit_Weight'].fillna(cf_test['Counterfeit_Weight'].mean(),inplace=True)

In [10]:
cf_train.isnull().sum()

Medicine_ID            0
Counterfeit_Weight     0
DistArea_ID            0
Active_Since           0
Medicine_MRP           0
Medicine_Type          0
SidEffect_Level        0
Availability_rating    0
Area_Type              0
Area_City_Type         0
Area_dist_level        0
Counterfeit_Sales      0
dtype: int64

In [11]:
cf_train.shape

(6818, 12)

In [12]:
cf_train['data']='train'
cf_test['data']='test'
cf_test['Counterfeit_Sales']=np.nan

In [13]:
cf_total=pd.concat([cf_train,cf_test])

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [14]:
cf_test.shape

(1705, 13)

In [15]:
cf_total.shape

(8523, 13)

In [16]:
cf_total['SidEffect_Level']=np.where(cf_total['SidEffect_Level']=='mild',0,1)

In [17]:
cf_total.dtypes

Active_Since             int64
Area_City_Type          object
Area_Type               object
Area_dist_level         object
Availability_rating    float64
Counterfeit_Sales      float64
Counterfeit_Weight     float64
DistArea_ID             object
Medicine_ID             object
Medicine_MRP           float64
Medicine_Type           object
SidEffect_Level          int32
data                    object
dtype: object

In [18]:
cf_total=cf_total.drop(['DistArea_ID','Medicine_ID'],axis=1)

In [19]:
objcol=['Area_City_Type','Area_Type','Area_dist_level','Medicine_Type']

In [20]:
for col in objcol:
    print(col+":"+str(cf_total[col].nunique())+str(cf_total[col].value_counts()))

Area_City_Type:3Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Area_City_Type, dtype: int64
Area_Type:4DownTown              5577
MidTownResidential    1083
CityLimits             935
Industrial             928
Name: Area_Type, dtype: int64
Area_dist_level:4Medium     2793
Unknown    2410
Small      2388
High        932
Name: Area_dist_level, dtype: int64
Medicine_Type:16Antibiotics           1232
Hreplacements         1200
Antiseptics            910
OralContraceptives     856
Antipyretics           682
Cardiac                649
Mstablizers            648
Tranquilizers          520
Analgesics             445
Antimalarial           425
Antacids               251
Statins                214
MuscleRelaxants        169
Antifungal             148
Stimulants             110
Antiviral               64
Name: Medicine_Type, dtype: int64


In [21]:
cat_cols=['Area_City_Type','Area_Type','Area_dist_level']

In [22]:
for col in cat_cols:
    freqs=cf_total[col].value_counts()
    k=freqs.index[freqs>400][:-1]
    for cat in k:
        name=col+'_'+cat
        cf_total[name]=(cf_total[col]==cat).astype(int)
    del cf_total[col]
    print(col)

Area_City_Type
Area_Type
Area_dist_level


In [23]:
def createdummies(x):
    k=cf_total[x].value_counts()
    for col in k.axes[0:15]:
        varname=x+"_"+str(k)
        cf_total[varname]=np.where(cf_total[x]==varname,1,0)
    del cf_total[x]   
        

In [24]:
createdummies('Medicine_Type')

In [25]:
cf_total

,Active_Since,Availability_rating,Counterfeit_Sales,Counterfeit_Weight,Medicine_MRP,SidEffect_Level,data,Area_City_Type_Tier 3,Area_City_Type_Tier 2,Area_Type_DownTown,Area_Type_MidTownResidential,Area_Type_CityLimits,Area_dist_level_Medium,Area_dist_level_Unknown,Area_dist_level_Small,"Medicine_Type_Antibiotics 1232\nHreplacements 1200\nAntiseptics 910\nOralContraceptives 856\nAntipyretics 682\nCardiac 649\nMstablizers 648\nTranquilizers 520\nAnalgesics 445\nAntimalarial 425\nAntacids 251\nStatins 214\nMuscleRelaxants 169\nAntifungal 148\nStimulants 110\nAntiviral 64\nName: Medicine_Type, dtype: int64"
0,1995,0.070422,1775.5026,13.100000,160.2366,1,train,0,0,1,0,0,0,0,1,0
1,1983,0.013000,3069.1520,14.115057,110.4384,0,train,1,0,0,0,1,1,0,0,0
2,1995,0.060783,2603.0920,9.025000,259.4092,0,train,0,0,1,0,0,0,0,1,0
3,1995,0.065555,1101.7130,11.800000,99.9830,0,train,0,0,1,0,0,0,0,1,0
4,1983,0.248859,158.9402,14.115057,56.4402,1,train,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1700,1983,0.050505,NaN,14.328604,136.5704,0,test,1,0,0,0,1,1,0,0,0
1701,2002,0.041118,NaN,21.300000,57.0744,1,test,0,1,1,0,0,0,0,1,0
1702,2005,0.191273,NaN,20.400000,182.7422,0,test,0,1,1,0,0,0,1,0,0
1703,2007,0.013000,NaN,20.000000,266.9672,0,test,1,0,0,0,0,1,0,0,0


In [26]:
cf_total.dtypes

Active_Since                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         int64
Availability_rating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [27]:
cf_train=cf_total[cf_total['data']=='train']
cf_test=cf_total[cf_total['data']=='test']


In [28]:
cf_train=cf_train.drop(['data'],1)
cf_test=cf_test.drop(['data','Counterfeit_Sales'],1)

In [29]:
cf_train.shape

(6818, 15)

In [30]:
cf_test.shape

(1705, 14)

In [31]:
train,val=train_test_split(cf_train,test_size=0.3,random_state=0)

In [32]:
train.shape

(4772, 15)

In [33]:
val.shape

(2046, 15)

In [34]:
y_train=train['Counterfeit_Sales']
train=train.drop(['Counterfeit_Sales'],1)

In [35]:
y_train

4105    2479.9190
6455     256.1470
1596    2339.4352
2278    1706.9252
1493     504.4904
          ...    
4931    5865.5120
3264     288.7712
1653    1515.1748
2607    1441.2710
2732    1314.1032
Name: Counterfeit_Sales, Length: 4772, dtype: float64

In [36]:
train.shape

(4772, 14)

In [37]:
y_val=val['Counterfeit_Sales']
val=val.drop(['Counterfeit_Sales'],1)

In [38]:
val.shape

(2046, 14)

In [39]:
rf=RandomForestRegressor(random_state=5)

In [40]:
rf.fit(train,y_train)

D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=5, verbose=0,
                      warm_start=False)

In [41]:
y_pred=rf.predict(val)

In [42]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_val,y_pred)
print('Mean Absolute Error : ',mae)

Mean Absolute Error :  808.4523034408602


In [43]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_val,y_pred)
rmse = np.sqrt(mse)
print('Root Mean Squared Error : ',rmse)

Root Mean Squared Error :  1157.829858369292


In [44]:
score=1-(mae/1660)

In [45]:
score

0.5129805400958674

In [46]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [47]:
import xgboost as xgb

In [48]:
model=xgb.XGBRegressor()

In [49]:
model.fit(train,y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [50]:
xgpredict=model.predict(val)

In [51]:
mae_xg = mean_absolute_error(y_val,xgpredict)
print('Mean Absolute Error : ',mae_xg)

Mean Absolute Error :  797.5972467956406


In [52]:
score1=(1-mae/1660)
print(score1)

0.5129805400958674


In [53]:
param_dist = {
              "max_depth": [2,3,4,5,6],
              "learning_rate":[0.01,0.05,0.1,0.3,0.5],
    "min_child_weight":[4,5,6],
              "subsample":[i/10.0 for i in range(6,10)],
 "colsample_bytree":[i/10.0 for i in range(6,10)],
               "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
              "gamma":[i/10.0 for i in range(0,5)],
    "n_estimators":[100,500,700,1000],
    'scale_pos_weight':[2,3,4,5,6,7,8,9]
    
              }

In [54]:
from sklearn.model_selection import RandomizedSearchCV

In [55]:
n_iter=10

random_search=RandomizedSearchCV(model,n_jobs=-1,verbose=2,cv=10,n_iter=n_iter,
                                 param_distributions=param_dist)

In [56]:
random_search.fit(train,y_train)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 10.4min finished


RandomizedSearchCV(cv=10, error_score='raise-deprecating',
                   estimator=XGBRegressor(base_score=0.5, booster=None,
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          gpu_id=-1, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=0.300000012,
                                          max_delta_step=0, max_depth=6,
                                          min_child_weight=1, missing=nan,
                                          monotone_constraints=None,
                                          n_...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.3,
                                  

In [57]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [58]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.590 (std: 0.03863)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 6, 'reg_alpha': 100, 'n_estimators': 100, 'min_child_weight': 4, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.7}

Model with rank: 2
Mean validation score: 0.590 (std: 0.04094)
Parameters: {'subsample': 0.9, 'scale_pos_weight': 5, 'reg_alpha': 100, 'n_estimators': 100, 'min_child_weight': 5, 'max_depth': 2, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.8}

Model with rank: 3
Mean validation score: 0.582 (std: 0.04230)
Parameters: {'subsample': 0.7, 'scale_pos_weight': 5, 'reg_alpha': 1, 'n_estimators': 700, 'min_child_weight': 4, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0.4, 'colsample_bytree': 0.6}

Model with rank: 4
Mean validation score: 0.571 (std: 0.04433)
Parameters: {'subsample': 0.8, 'scale_pos_weight': 9, 'reg_alpha': 1, 'n_estimators': 500, 'min_child_weight': 4, 'max_depth': 3, 'learning_rate': 0.05, 'gamma': 0.4,

In [59]:
xbg_best=xgb.XGBRegressor(subsample= 0.9, scale_pos_weight= 6, reg_alpha= 0.01, 
                          n_estimators= 100, min_child_weight= 6, max_depth= 3, learning_rate= 0.05, 
                          gamma= 0.3, colsample_bytree= 0.9)

In [60]:
xbg_best.fit(train,y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0.3, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=0, max_depth=3,
             min_child_weight=6, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0.01,
             reg_lambda=1, scale_pos_weight=6, subsample=0.9, tree_method=None,
             validate_parameters=False, verbosity=None)

In [61]:
a=xbg_best.predict(val)

In [62]:
mae_xgb_best = mean_absolute_error(y_val,a)
print('Mean Absolute Error : ',mae_xgb_best)

Mean Absolute Error :  730.9028275147798


In [63]:
score1=(1-mae_xgb_best/1660)
print(score1)

0.5596970918585664


In [65]:
test_pred=xbg_best.predict(cf_test)

In [66]:
submission=pd.DataFrame(list(zip(cf_test1['Medicine_ID'],list(test_pred))),
                       columns=['Medicine_ID','Counterfeit_Sales'])

In [67]:
submission.to_csv('Counterfeit_Predict1.csv',index=False)